In [ ]:
import sys
import os
from importlib import reload
fpath = os.path.join('..//scripts')
sys.path.append(fpath)

import warnings
warnings.filterwarnings('ignore')

#loading internal scripts
import frauddetection as fd
import sourcedata as sd
import countrymanagement as cm
import mccmanagement as mccm
reload(fd)
reload(sd)
reload(cm)
reload(mccm)

print('done')

In [ ]:
import pandas as pd

#For Kaggle
#date=''

#For WL data
date='20241118'
source='WL'

saveImg=False

print('done')

dfTrxSaved = pd.read_csv('../data/cleaned/'+source+'export'+date+'.csv')
dfTrxSaved.head(5)

In [ ]:
sorted_df = dfTrxSaved.sort_values(by=['card_pan_id','trx_date_time'])
sorted_df=sorted_df.drop(columns=['mcd_fraud_score','vaa_score',
                                 'trx_amount_log10','ecom',	'card_brand_MCD',	'card_brand_VIS','country_group_BELGIUM','country_group_EUROPE',
                                 'clusterCardHolder_cluster0',	'clusterCardHolder_cluster1',	'clusterCardHolder_cluster2',	'clusterCardHolder_cluster3',	
                                 'clusterCardHolder_cluster4',	'clusterMerchant_M0',	'clusterMerchant_M1','clusterMerchant_M2','clusterMerchant_M3',
                                 'clusterMerchant_UNKNOWN',
                                  'country_group_WORLD','mcc_group_CLOTHING_STORES',	
                                  'mcc_group_GOUVERNEMENT_SERVICES','mcc_group_OTHER','mcc_group_PROFESSIONAL_SERVICES',	
                                  'mcc_group_RETAIL_OUTLET_SERVICES','mcc_group_UTILITY_SERVICES','mcc_group_ATM',	
                                  'trx_reversal_FULL_REVERSAL',	'trx_reversal_NO_RESERSAL',	
                                  'trx_reversal_PARTIAL_REVERSAL','clusterCardHolder_UNKNOWN'
                                 ])
#print(sorted_df.head(10))
sorted_df['mcc_group'] = sorted_df['term_mcc'].apply(mccm.get_mcc_group_citybank)
sorted_df['mcc_group'] = np.where(sorted_df.term_mcc.isin([mccm.get_mcc_group_ATM]),'ATM',sorted_df['mcc_group'] )
sorted_df['mcc_group'] = np.where(sorted_df.mcc_group.isin(['AGRICULTURAL']), 'OTHER',sorted_df['mcc_group'] )
sorted_df['mcc_group'] = np.where(sorted_df.mcc_group.isin(['CONTRACTED_SERVICES']), 'OTHER',sorted_df['mcc_group'] )
sorted_df['mcc_group'] = np.where(sorted_df.mcc_group.isin(['AIRLINES']),'OTHER',sorted_df['mcc_group'] )
sorted_df['mcc_group'] = np.where(sorted_df.mcc_group.isin(['CAR_RENTAL']), 'OTHER',sorted_df['mcc_group'] )
sorted_df['mcc_group'] = np.where(sorted_df.mcc_group.isin(['LODGING']),'OTHER',sorted_df['mcc_group'] )
sorted_df['mcc_group'] = np.where(sorted_df.mcc_group.isin(['TRANSPORTATION_SERVICES']), 'OTHER',sorted_df['mcc_group'] )
sorted_df['mcc_group'] = np.where(sorted_df.mcc_group.isin(['MISCELLANOUS_STORES']), 'OTHER',sorted_df['mcc_group'] )
sorted_df['mcc_group'] = np.where(sorted_df.mcc_group.isin(['BUSINESS_SERVICES']), 'OTHER',sorted_df['mcc_group'] )

sorted_df['term_mcc'] = sorted_df['term_mcc'].astype('str')

sorted_df['card_pan_id1'] = sorted_df['card_pan_id'].shift(-1)
#sorted_df['term_mcc1']= sorted_df['term_mcc'].shift(-1) if sorted_df['card_pan_id1'] == sorted_df['card_pan_id'] else 'XXXX'
sorted_df['term_mcc1'] = sorted_df['term_mcc'].shift(-1)
sorted_df['mcc_group1'] = sorted_df['mcc_group'].shift(-1)
sorted_df['term_country1'] = sorted_df['term_country'].shift(-1)
sorted_df['db_uuid1'] = sorted_df['db_uuid'].shift(-1)
sorted_df.head(10)


In [ ]:
sorted_df1=sorted_df[(sorted_df['card_pan_id']==sorted_df['card_pan_id1'])]
sorted_df0=sorted_df[(sorted_df['card_pan_id']!=sorted_df['card_pan_id1'])]

print(sorted_df.shape)
print(sorted_df0.shape)
print(sorted_df1.shape)

print(sorted_df.Class.value_counts(normalize=True))
print(sorted_df0.Class.value_counts(normalize=True))
print(sorted_df1.Class.value_counts(normalize=True))


#sorted_df4.head(50)

In [ ]:
import numpy as np

dfTrxSaved['previous_trx']=0

sorted_dfTemp=sorted_df[(sorted_df['card_pan_id']==sorted_df['card_pan_id1'])]
dfTrxSaved['previous_trx']=np.where(sorted_df.index.isin(sorted_dfTemp.index),1,dfTrxSaved['previous_trx'])
#dfTrxSaved['term_mcc1']=np.where(sorted_df.index.isin(sorted_dfTemp.index),dfTrxSaved['term_mcc1'],'')
#dfTrxSaved['term_country1']=np.where(sorted_df.index.isin(sorted_dfTemp.index),dfTrxSaved['term_country1'],'')

dfTrxSaved['card_pan_id1']=sorted_df['card_pan_id1']

dfTrxSaved['term_mcc1']=sorted_df['term_mcc1']
dfTrxSaved['mcc_group']=sorted_df['mcc_group']
dfTrxSaved['mcc_group1']=sorted_df['mcc_group1']
dfTrxSaved['term_country1']=sorted_df['term_country1']
dfTrxSaved['db_uuid1']=sorted_df['db_uuid1']

dfTrxSaved['previous_trx'].value_counts()


In [ ]:
dfTrxSaved['country_group'] = dfTrxSaved['term_country'].apply(cm.get_country_group)
dfTrxSaved['country_group1'] = dfTrxSaved['term_country1'].apply(cm.get_country_group)

dfTrxSaved['term_mcc1'].fillna('99999',inplace=True)
dfTrxSaved['term_mcc'] = dfTrxSaved['term_mcc'].astype('int')
dfTrxSaved['term_mcc1'] = dfTrxSaved['term_mcc1'].astype('int')


dfTrxSaved[['country_group','country_group1','mcc_group','mcc_group1']].head(5)

In [ ]:
print(f'IV previous_trx  {fd.calc_iv(dfTrxSaved,'previous_trx','Class',pr=0)}')

In [ ]:


temp=dfTrxSaved[(dfTrxSaved['Class']==1)]

temp[['previous_trx','card_pan_id1','card_pan_id','term_country','term_country1', 'Class']].head(5)

In [ ]:
for i in range(0,3,1):
    print(i)
    print(dfTrxSaved[(dfTrxSaved['previous_trx']==i)].shape[0])
    print(dfTrxSaved['Class'][(dfTrxSaved['previous_trx']==i)].value_counts(normalize=True))

In [ ]:
dfTrxSaved[(dfTrxSaved['previous_trx']==1)][['term_country','term_country1','term_mcc','term_mcc1']].head()

In [ ]:
dfTrxSaved[(dfTrxSaved['previous_trx']==1)][['term_country','term_country1','term_mcc','term_mcc1']].head()

In [ ]:
dfTrxSaved['term_mcc'] = dfTrxSaved['term_mcc'].astype('str')
dfTrxSaved.info()

In [ ]:
dfTrxSaved['distancePrevTrx']=1.0 #cosinus distance ?

In [ ]:
dfTemp=dfTrxSaved[(dfTrxSaved['previous_trx']==1) 
    & (dfTrxSaved['term_mcc']==dfTrxSaved['term_mcc1']) 
    & (dfTrxSaved['term_country']==dfTrxSaved['term_country1']) ]
dfTrxSaved['distancePrevTrx']=np.where(sorted_df.index.isin(dfTemp.index),1,dfTrxSaved['distancePrevTrx'])

dfTemp=dfTrxSaved[(dfTrxSaved['previous_trx']==1) 
    & (dfTrxSaved['term_mcc']==dfTrxSaved['term_mcc1']) 
    & (dfTrxSaved['term_country']!=dfTrxSaved['term_country1']) ]
dfTrxSaved['distancePrevTrx']=np.where(sorted_df.index.isin(dfTemp.index),0.4,dfTrxSaved['distancePrevTrx'])

dfTemp=dfTrxSaved[(dfTrxSaved['previous_trx']==1) 
    & (dfTrxSaved['term_mcc']!=dfTrxSaved['term_mcc1']) 
    & (dfTrxSaved['term_country']==dfTrxSaved['term_country1']) ]
dfTrxSaved['distancePrevTrx']=np.where(sorted_df.index.isin(dfTemp.index),0.6,dfTrxSaved['distancePrevTrx'])

dfTemp=dfTrxSaved[(dfTrxSaved['previous_trx']==1) 
    & (dfTrxSaved['term_mcc']!=dfTrxSaved['term_mcc1']) 
    & (dfTrxSaved['term_country']!=dfTrxSaved['term_country1']) ]
dfTrxSaved['distancePrevTrx']=np.where(sorted_df.index.isin(dfTemp.index),0,dfTrxSaved['distancePrevTrx'])

print(f'IV previous_trx  {fd.calc_iv(dfTrxSaved,'distancePrevTrx','Class',pr=0)}')
print("1.0 ",dfTrxSaved[(dfTrxSaved['distancePrevTrx']==1.0)].Class.value_counts(normalize=True))
print("0.6 ",dfTrxSaved[(dfTrxSaved['distancePrevTrx']==0.6)].Class.value_counts(normalize=True))
#print("0.4 ",dfTrxSaved[(dfTrxSaved['distancePrevTrx']==0.4)].Class.value_counts(normalize=True))
print("0.0 ",dfTrxSaved[(dfTrxSaved['distancePrevTrx']==0.0)].Class.value_counts(normalize=True))
dfTrxSaved['distancePrevTrx'].value_counts()

In [ ]:
dfTemp=dfTrxSaved[(dfTrxSaved['previous_trx']==1) 
    & (dfTrxSaved['mcc_group']==dfTrxSaved['mcc_group1']) 
    & (dfTrxSaved['country_group']==dfTrxSaved['country_group1']) ]
dfTrxSaved['distancePrevTrx']=np.where(sorted_df.index.isin(dfTemp.index),1,dfTrxSaved['distancePrevTrx'])

dfTemp=dfTrxSaved[(dfTrxSaved['previous_trx']==1) 
    & (dfTrxSaved['mcc_group']==dfTrxSaved['mcc_group1']) 
    & (dfTrxSaved['country_group']!=dfTrxSaved['country_group1']) ]
dfTrxSaved['distancePrevTrx']=np.where(sorted_df.index.isin(dfTemp.index),0.4,dfTrxSaved['distancePrevTrx'])

dfTemp=dfTrxSaved[(dfTrxSaved['previous_trx']==1) 
    & (dfTrxSaved['mcc_group']!=dfTrxSaved['mcc_group1']) 
    & (dfTrxSaved['country_group']==dfTrxSaved['country_group1']) ]
dfTrxSaved['distancePrevTrx']=np.where(sorted_df.index.isin(dfTemp.index),0.6,dfTrxSaved['distancePrevTrx'])

dfTemp=dfTrxSaved[(dfTrxSaved['previous_trx']==1) 
    & (dfTrxSaved['mcc_group']!=dfTrxSaved['mcc_group1']) 
    & (dfTrxSaved['country_group']!=dfTrxSaved['country_group1']) ]
dfTrxSaved['distancePrevTrx']=np.where(sorted_df.index.isin(dfTemp.index),0,dfTrxSaved['distancePrevTrx'])


print(f'IV previous_trx  {fd.calc_iv(dfTrxSaved,'distancePrevTrx','Class',pr=0)}')
print("1.0 ",dfTrxSaved[(dfTrxSaved['distancePrevTrx']==1.0)].Class.value_counts(normalize=True))
print("0.6 ",dfTrxSaved[(dfTrxSaved['distancePrevTrx']==0.6)].Class.value_counts(normalize=True))
print("0.4 ",dfTrxSaved[(dfTrxSaved['distancePrevTrx']==0.4)].Class.value_counts(normalize=True))
print("0.0 ",dfTrxSaved[(dfTrxSaved['distancePrevTrx']==0.0)].Class.value_counts(normalize=True))
dfTrxSaved['distancePrevTrx'].value_counts()